In [ ]:
#Instalar estas librerias 
!pip install nltk
!pip install stanza


In [ ]:
#Una vez instaladas, bajar los siguientes paquetes de esas librerias (ejecutar unicamente la primera vez)
import nltk

nltk.download('punkt')
nltk.download('stopwords')
stanza.download('en')

In [ ]:
import numpy as np
import pandas as pd

import stanza


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import pickle

import matplotlib.pyplot as plt

# Funciones utiles: ejecutar todas

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')
#Documentacion stanza: https://stanfordnlp.github.io/stanza/lemma.html
#Toma como imput un string crudo porq lematiza por contexto


# Importar datos

In [ ]:
#Definir pandas dataframe
df = pd.read_csv('')

# Procesamiento de mensajes

In [ ]:
from utils import procesar_articulos

In [ ]:
df_procesado = procesar_articulos((df.reset_index())['columna_de_contenido'], nlp, language = 'spanish')
#Esta funcion toma como imput un pd.series de strings. 
#poner la columna con las encuestas nada mas
#IMPORTANTE: no borrar el reset_index

# LDA

Definir solo una vez

In [ ]:
from plot_helper import plot_top_words, get_keys, keys_to_counts

In [ ]:
count_vectorizer = CountVectorizer(max_df = 0.8, min_df = 10) 
#si el vocabulario te queda chico, podes bajar el parametro "min_df". Lo que hace es tomar unicamente las palabras 
#q aparecen en mas de n documentos. En este caso, 10. No lo bajes tanto para evitar que aparezca mucha basura

In [ ]:
vectorized_df = count_vectorizer.fit_transform(df_procesado)

Ahora ejecutas el LDA, lo de arriba no hace falta volver a definirlo para cambiar la cantidad de clusters

In [ ]:
n_topics = 4    #cantidad de clusters. cambiar desde aca.

In [ ]:
lda_model = LatentDirichletAllocation(n_components=n_topics,
                                          random_state=42) 

In [ ]:
lda_topic_matrix = lda_model.fit_transform(vectorized_df)

In [ ]:
#Aca te aparecen las palabras mas importantes por cluster

vocab = count_vectorizer.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20] #El 20 es la cantidad de palabras q aparecen
    print("Topic "+str(i+1)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

In [ ]:
lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

In [ ]:
plot_top_words(lda_model, count_vectorizer.get_feature_names(), 20, 'Top words LDA clustering', cols = int(n_topics/2))
#El 20 es la cantidad de palabras que queres que aparezcan por cluster.

In [ ]:
labels = ['Topic '+str(i+1)+ '\n'.format(i) for i in lda_categories]

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lda_categories, lda_counts);
ax.set_xticks(lda_categories);
ax.set_xticklabels(labels);
ax.set_ylabel('Cantidad de reviews');
ax.set_title('Cantidad de reviews por cluster LDA');
plt.show()